In [1]:
import random
from sklearn import datasets, linear_model
from sklearn import metrics

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import Normalizer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

In [10]:
def read_data(run_num):
    data = np.genfromtxt("datasets/iris/pima-indians-diabetes.csv", delimiter=",")
    data_X = data[:, :8]   # all features 1-7
    
    # transformer = Normalizer().fit(data_X)
    # data_X = transformer.transform(data_X)
    
    data_y = data[:, -1]   # target
    
    # percent_test = 0.4
    # testsize = int(percent_test * data_X.shape[0])
    # X_train = data_X[:-testsize]
    # X_test = data_X[-testsize:]
    # y_train = data_y[:-testsize]
    # y_test = data_y[-testsize:]
    
    X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.4, random_state=run_num)
    
    return X_train, X_test, y_train, y_test

def scipy_linear_mod(X_train, X_test, y_train, y_test, type_model):
    if type_model == 0:
        log_reg = linear_model.LogisticRegression(tol=0.1, solver='saga')
    elif type_model == 1:
        log_reg = linear_model.LogisticRegression(tol=0.1, solver='saga', penalty='l1')
    else:
        log_reg = linear_model.LogisticRegression(tol=0.1, solver='saga', penalty='l2')
        
        # saga: Stochastic Average Gradient descent 
        # tol: Tolerance for stopping criteria
        
    # Train the model
    log_reg.fit(X_train, y_train)
    
    # Make prediction
    y_pred = log_reg.predict(X_test)
    #print("y_pred: \n", y_pred)
    
    print("\nCoefficients: \n", log_reg.coef_)
    print("RMSE: %.2f" % mean_squared_error(y_pred, y_test, squared=False))
    print("Explained variance score: %.2f" % r2_score(y_test, y_pred))
    
    acc = accuracy_score(y_pred, y_test)
    print("Accuracy score: ", acc)
    
    auc = roc_auc_score(y_pred, y_test, average=None)
    print("AUC score: ", auc)
    
    corr_mat = confusion_matrix(y_pred, y_test)
    print("Confusion matrix: \n", corr_mat)
    
    #Rather than passing the class predictions (y_pred), to pass the probabilities
    y_pred_ = log_reg.predict_proba(X_test)[:, -1]
    
    # Otherwise
    #y_pred_ = log_reg.decision_function(X_test)
    
    lr_fpr, lr_tpr, _ = roc_curve(y_test, y_pred_)
    
    # Plot the ROC curve for the method 1
    plt.plot(lr_fpr, lr_tpr, marker='.', label='Logistic-model')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.savefig('figures/'+str(type_model)+'plot.png')
    plt.clf()
    
    # Cross validation
    cv_results = cross_validate(log_reg, X_train, y_train, cv=10)
    print("10-fold Cross Validation: ", cv_results)
    
    return auc, acc

def main():
    max_expruns = 3
    accbase_all = np.zeros(max_expruns)
    accl1_all = np.zeros(max_expruns)
    accl2_all = np.zeros(max_expruns)
    
    for run_num in range(max_expruns):
        X_train, X_test, y_train, y_test = read_data(run_num)
        auc_base, acc_base = scipy_linear_mod(X_train, X_test, y_train, y_test, 0)
        auc_l1, acc_l1 = scipy_linear_mod(X_train, X_test, y_train, y_test, 1)
        auc_l2, acc_l2 = scipy_linear_mod(X_train, X_test, y_train, y_test, 2)
        
        accbase_all[run_num] = acc_base
        accl1_all[run_num] = acc_l1
        accl2_all[run_num] = acc_l2
        
        print('-- run num --', run_num)
        print('acc_base, acc_l1, acc_l2', acc_base, acc_l1, acc_l2)
    
    print('\naccbase_all', accbase_all)
    print('mean accbase_all', np.mean(accbase_all))
    print('std accbase_all', np.std(accbase_all))
    
    print('accl1_all', accl1_all)
    print('mean accl1_all', np.mean(accl1_all))
    print('std accl1_all', np.std(accl1_all))
    

if __name__ == '__main__':
    main()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Coefficients: 
 [[ 6.79917033e-04  5.42756385e-03 -1.39193074e-02 -3.33983272e-03
   1.27651284e-03 -2.40839037e-03 -3.86565901e-06 -1.70212256e-03]]
RMSE: 0.58
Explained variance score: -0.50
Accuracy score:  0.6655844155844156
AUC score:  0.5917266187050358
Confusion matrix: 
 [[190  88]
 [ 15  15]]


...

<IPython.core.display.Javascript object>

mean accl1_all 0.6504329004329005


<IPython.core.display.Javascript object>

std accl1_all 0.021590841280584423


<Figure size 432x288 with 0 Axes>